## <center>Підготовка та аналіз даних</center>
## <center>Лабораторна робота №2</center>
### <center>ФБ-32 Чупріна Вікторія </center>

Імпортуємо потрібні нам бібліотеки

In [1]:
import datetime
import requests
import urllib.request
import pandas as pd
import os


Скачуємо VHI індекси для всіх областей
Також додаємо в назву дату та час скачування

In [2]:
for ids in range(1, 28):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={ids}&year1=1981&year2=2024&type=Mean"
    response = requests.get(url)
    if response.status_code == 200:
        if not os.path.exists('vhi'):
            os.mkdir('vhi')
            print(f'The folder is created')
        date_now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        vhi_url = urllib.request.urlopen(url)
        file_name = f'vhi/vhi_id_{ids}_{date_now}.csv'
        out = open(file_name, 'wb')
        out.write(vhi_url.read())
        out.close()
        print(f"VHI from id {ids} was downloaded at {date_now}")
    else:
        print(f"Download complete")
        break

The folder is created
VHI from id 1 was downloaded at 2025-03-13_12-39-55
VHI from id 2 was downloaded at 2025-03-13_12-39-57
VHI from id 3 was downloaded at 2025-03-13_12-40-00
VHI from id 4 was downloaded at 2025-03-13_12-40-03
VHI from id 5 was downloaded at 2025-03-13_12-40-05
VHI from id 6 was downloaded at 2025-03-13_12-40-07
VHI from id 7 was downloaded at 2025-03-13_12-40-09
VHI from id 8 was downloaded at 2025-03-13_12-40-12
VHI from id 9 was downloaded at 2025-03-13_12-40-14
VHI from id 10 was downloaded at 2025-03-13_12-40-16
VHI from id 11 was downloaded at 2025-03-13_12-40-18
VHI from id 12 was downloaded at 2025-03-13_12-40-20
VHI from id 13 was downloaded at 2025-03-13_12-40-23
VHI from id 14 was downloaded at 2025-03-13_12-40-25
VHI from id 15 was downloaded at 2025-03-13_12-40-27
VHI from id 16 was downloaded at 2025-03-13_12-40-30
VHI from id 17 was downloaded at 2025-03-13_12-40-32
VHI from id 18 was downloaded at 2025-03-13_12-40-35
VHI from id 19 was downloaded at 

Даємо імена стовбцям, зчитуємо файли у фрейм

In [ ]:
folder = 'vhi'
files = os.listdir(folder)

df_all = []

for file_name in files:
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    df = pd.read_csv(f'{folder}/{file_name}', header=1, names=headers, skiprows=1)
    df = df.drop(columns=['empty'], errors='ignore')

    df['VHI'] = pd.to_numeric(df['VHI'], errors='coerce')
    df = df[df['VHI'] != -1].dropna()

    parts = file_name.split("_")
    if len(parts) > 2 and parts[2].isdigit():
        df['area'] = int(parts[2])
    else:
        print(f"Попередження: неможливо оприділити 'area' для файла {file_name}")
        continue 

    df_all.append(df)

df_all = pd.concat(df_all, ignore_index=True)

df_all = df_all.dropna(axis=1, how='all')

print(df_all)

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   2.0  0.063  261.53  55.89  38.20  47.04    10
1      1982   3.0  0.063  263.45  57.30  32.69  44.99    10
2      1982   4.0  0.061  265.10  53.96  28.62  41.29    10
3      1982   5.0  0.058  266.42  46.87  28.57  37.72    10
4      1982   6.0  0.056  267.47  39.55  30.27  34.91    10
...     ...   ...    ...     ...    ...    ...    ...   ...
58990  2024  48.0  0.135  278.17  55.23  11.86  33.55     9
58991  2024  49.0  0.133  277.08  57.71  10.86  34.29     9
58992  2024  50.0  0.130  276.49  59.45   8.68  34.07     9
58993  2024  51.0  0.128  276.45  62.53   5.55  34.04     9
58994  2024  52.0  0.129  276.48  66.13   3.71  34.92     9

[58995 rows x 8 columns]


Змінимо індекси областей

In [4]:
dict_areas = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12,
              16: 13, 17: 15, 18: 14, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
df_all["area"] = df_all["area"].replace(dict_areas)
df_all.to_csv(f'vhi/df_all.csv', index=False)

print(df_all)

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   2.0  0.063  261.53  55.89  38.20  47.04    21
1      1982   3.0  0.063  263.45  57.30  32.69  44.99    21
2      1982   4.0  0.061  265.10  53.96  28.62  41.29    21
3      1982   5.0  0.058  266.42  46.87  28.57  37.72    21
4      1982   6.0  0.056  267.47  39.55  30.27  34.91    21
...     ...   ...    ...     ...    ...    ...    ...   ...
58990  2024  48.0  0.135  278.17  55.23  11.86  33.55    20
58991  2024  49.0  0.133  277.08  57.71  10.86  34.29    20
58992  2024  50.0  0.130  276.49  59.45   8.68  34.07    20
58993  2024  51.0  0.128  276.45  62.53   5.55  34.04    20
58994  2024  52.0  0.129  276.48  66.13   3.71  34.92    20

[58995 rows x 8 columns]


Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [15]:
def vhi(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]['VHI']

def vhi_min(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]['VHI'].min()

def vhi_max(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]['VHI'].max()

df_all['Year'] = df_all['Year'].astype(int)

In [6]:
vhi(6, 2005)

31736    47.42
31737    51.20
31738    53.68
31739    54.35
31740    54.66
31741    56.10
31742    56.81
31743    56.06
31744    56.27
31745    56.51
31746    56.42
31747    56.02
31748    54.84
31749    52.12
31750    49.52
31751    48.44
31752    53.62
31753    52.32
31754    52.21
31755    51.77
31756    53.50
31757    54.49
31758    54.25
31759    55.50
31760    57.29
31761    57.93
31762    58.44
31763    65.13
31764    59.35
31765    61.40
31766    63.51
31767    64.39
31768    66.13
31769    65.04
31770    60.86
31771    57.68
31772    56.59
31773    54.98
31774    53.23
31775    52.48
31776    55.20
31777    57.22
31778    57.66
31779    58.95
31780    56.50
31781    54.77
31782    54.48
31783    53.22
31784    50.36
31785    47.41
31786    45.59
Name: VHI, dtype: float64

In [7]:
vhi_min(6, 2005)


45.59

In [8]:
vhi_max(6, 2005)

66.13

Ряд VHI за вказаний діапазон років для вказаних областей;

In [9]:
def vhi_range(year_min, year_max, areas):
    if not isinstance(areas, list) or not areas:
        return print('Empty or not a list')
    return df_all[(df_all['Year'] >= year_min) & (df_all['Year'] <= year_max) & (df_all['area'].isin(areas))][['Year', 'VHI', 'area']]

In [10]:
vhi_range(2000, 2005, [5, 2, 3])

,Year,VHI,area
35865,2000,24.65,2
35866,2000,27.49,2
35867,2000,31.36,2
35868,2000,37.28,2
35869,2000,40.85,2
...,...,...,...
49262,2005,40.70,3
49263,2005,42.92,3
49264,2005,41.13,3
49265,2005,41.49,3


Роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей

In [11]:
def extreme_droughts(procent):
    df_drouhts = df_all[(df_all['VHI'] <= 15) & (df_all['VHI'] != -1)]
    grouped = df_drouhts.groupby('Year')['area'].nunique()
    result = grouped[grouped > (25 * procent / 100)].reset_index()
    return result

In [12]:
extreme_droughts(5)

,Year,area
0,1986,2
1,1993,2
2,2000,6
3,2007,5


Аналогічно для помірних посух

In [13]:
def moderate_droughts(procent, min=15, max=40):
    df_drouhts = df_all[(df_all['VHI'] >= min) & (df_all['VHI'] <= max)]
    grouped = df_drouhts.groupby('Year')['area'].nunique()
    count_areas = df_all['area'].nunique()
    result = grouped[grouped > (count_areas * procent / 100)].reset_index()
    return result

In [14]:
moderate_droughts(20)

,Year,area
0,1982,25
1,1983,25
2,1984,25
3,1985,25
4,1986,25
5,1987,25
6,1988,25
7,1989,25
8,1990,25
9,1991,24
